In [1]:
%reset -f
import gc
gc.collect()

0

In [2]:
import numpy as np
import pandas as pd
import pymysql
import csv
import os
import datetime
from opencc import OpenCC
import re
import chardet
import random
import time
import json
from tqdm import trange
import requests
from bs4 import BeautifulSoup

## 連線與建資料庫，有建立過不用再建

In [3]:
# #連線
# Database = pymysql.connect(host="localhost", user="root", password="mysql")
# cursor = Database.cursor()
# cursor.execute("drop database if exists cnyes_news") #刪資料庫
# cursor.execute("create database cnyes_news character set utf8 collate utf8_general_ci") #建資料庫
#登入
Database = pymysql.connect(host="localhost", user="root", password="mysql", db="cnyes_news")
Cursor=Database.cursor()
Cursor.execute("use cnyes_news")

0

## 建立table，有建立過不用再建

In [6]:
# # for TableName in ['tw_stock','wd_stock','cn_stock','forex','future','cnyeshouse']:
# for TableName in ['tw_stock']:
#     #刪除表
# #     Cursor.execute("drop table if exists %s"%(TableName))
#     #建立表
#     Cursor.execute("drop table if exists %s"%(TableName))
#     BuiltQuery = """CREATE TABLE %s (ID bigint(10) NOT NULL AUTO_INCREMENT PRIMARY KEY,
#                              Created TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
#                              Datatime bigint(12),
#                              Newstime bigint(12),
#                              Source varchar(1000) CHARACTER SET utf8 COLLATE utf8_general_ci,
#                              Header varchar(1000) CHARACTER SET utf8 COLLATE utf8_general_ci,
#                              Content text CHARACTER SET utf8 COLLATE utf8_general_ci,
#                              Target varchar(1000) CHARACTER SET utf8 COLLATE utf8_general_ci,
#                              URL varchar(1000) CHARACTER SET utf8 COLLATE utf8_general_ci)""" %(TableName)
#     Cursor.execute(BuiltQuery)

C:\Users\jimmy\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)
C:\Users\jimmy\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3778, "'utf8_general_ci' is a collation of the deprecated character set UTF8MB3. Please consider using UTF8MB4 with an appropriate collation instead.")
  result = self._query(query)


#### api格式 https://news.cnyes.com/api/v3/news/category/{專欄分類}?startAt={開始日期}&endAt={結束日期}&limit=30

In [4]:
# for category in ['tw_stock','wd_stock','cn_stock','forex','future','cnyeshouse']:  
for category in ['tw_stock']:  
    print(category)
    FindQuery = """select max(Newstime) from %s""" %(category) #找出目前資料庫最新日期(a)
    Cursor.execute(FindQuery)
    MaxNewstime=str(Cursor.fetchall()[0][0])[0:8] #把cursor資料全抓出，[0][0]剛好取出數字(int)，然後轉成str，取日期共8位數
    MaxNewstime = time.mktime(time.strptime(MaxNewstime, "%Y%m%d"))#轉成時間戳
    Nowtime = time.mktime(time.strptime(datetime.datetime.now().strftime("%Y-%m-%d"),"%Y-%m-%d")) #執行程式日期(b)，一樣轉成時間戳
    Days = (Nowtime-MaxNewstime)/(60*60*24) #時間戳互減，並轉換成天數格式
    
    for i3 in range(int(Days-1)): #爬取(a+1)至(b-1) 
        #重點抓出max page
        bench_time = int(MaxNewstime + 60*60*24*1) #現有資料庫最新日期(a)隔天(a+1)
        start_time = bench_time + 60*60*24*i3 #從a+1到b-1
        link1="https://news.cnyes.com/api/v3/news/category/%s?startAt=%s&endAt=%s&limit=30"%(category,str(start_time),str(start_time)) #注意catergory、startAt、endAt
        reqs = requests.get(link1)
        reqsjson = json.loads(reqs.text)
        max_page = reqsjson['items']['last_page']
        
        for i2 in range(max_page): 
            #重點抓出每頁js內容
            link2="https://news.cnyes.com/api/v3/news/category/%s?startAt=%s&endAt=%s&limit=30&page=%s"%(category,str(start_time),str(start_time),i2+1)
            reqs = requests.get(link2)
            reqsjson = json.loads(reqs.text)

            for i1 in range(len(reqsjson['items']['data'])): #第N篇新聞，前面有設定limit30
                TableName=category #資料庫table
                datatime=datetime.datetime.now().strftime('%Y%m%d%H%M%S')[0:12] #爬蟲抓新聞的時間
                Newstime=time.strftime('%Y%m%d%H%M%S',time.localtime(reqsjson['items']['data'][i1]['publishAt']))[0:12] #抓出新聞發布時間
                Source=reqsjson['items']['data'][i1]['categoryId']
                Header=reqsjson['items']['data'][i1]['title'].replace("\n"," ").replace("\\","\\\\").replace('"',r'\"')
                
                #抓出標的公司
                Target=''
                for i0 in range(len(reqsjson['items']['data'][i1]['market'])): #抓出標的公司名稱與代碼
                    tmpp=reqsjson['items']['data'][i1]['market'][i0]['name']+reqsjson['items']['data'][i1]['market'][i0]['code']
                    Target=Target+tmpp+','  #可能不只一個，中間用逗號隔開
                if Target != "":
                    Target=Target[:-1] #不留下最後一個逗號
                    
                url="https://news.cnyes.com/news/id/"+str(reqsjson['items']['data'][i1]['newsId'])#新聞網址
                response = requests.get(url)
                soup = BeautifulSoup(response.text,'lxml')
                con = soup.find_all("p")
                
                Content = ""
                #這段會多抓出幾個字
#                 for i0 in range(len(con)):
#                     tmp = con[i0].get_text()
#                     Content = Content + tmp
#                 Content="".join(Content.split()).replace("\n"," ").replace("\\","\\\\").replace('"',r'\"')
                
                if Content =='':
                    con = soup.find_all('div',{"itemprop":"articleBody"})
                    for i0 in range(len(con)):
                        tmp = con[i0].get_text()
                        Content = Content + tmp
                    Content="".join(Content.split()).replace("\n"," ").replace("\\","\\\\").replace('"',r'\"')
#                 if len(Content)>=50000:
#                     Content=""
#                     print("太長")
                
                
                #InsertQuery
                InsertQuery = """insert into %s(datatime, Newstime, Source, Header, url, Target, Content)
                                values(%d,%d,"%s","%s","%s","%s","%s")
                                """%(TableName, int(datatime), int(Newstime), Source, Header, url, Target, Content)
                
                try:
                    Cursor.execute(InsertQuery)
                    Database.commit()
                except :
                    print(url,"Error")
                
Cursor.close()
Database.close()

tw_stock
